In [1]:
import numpy as np              # 모듈 불러오기
from scipy.stats import norm    # 모듈 불러오기

class EM:           # class 명 EM으로 정의
    def __init__(self,x1_list,x2_list):         # 받는 인자값은 2가지
        self.x1_list = x1_list              # 받은 인자값을 클래스 변수에 저장
        self.x2_list = x2_list              # 받은 인자값을 클래스 변수에 저장
        self.x_list = np.concatenate((self.x1_list, self.x2_list), axis=None)         # 받은 두개의 인자값(리스트)을 합침
        self.mu_sigma_prob = [[7,3,0.5],[20,3,0.5]]     # mu_sigma_prob 순. 즉, 평균, 표준편차, 가중치 순으로 리스트에 저장 여기서 확률밀도함수는 2개 이므로 2x3 배열로 저장
        self.c1 = norm(loc=self.mu_sigma_prob[0][0], scale=self.mu_sigma_prob[0][1])        # self.mu_sigma_prob의 값을 불러와서 정규분포의 객체 생성 후 self.c1에 저장
        self.c2 = norm(loc=self.mu_sigma_prob[1][0], scale=self.mu_sigma_prob[1][1])        # self.mu_sigma_prob의 값을 불러와서 정규분포의 객체 생성 후 self.c2에 저장
        self.cluster = [[],[]]              # cluster를 저장할 배열 초기화
        print('모든 점들의 좌표',self.x_list)      # 받은 점들의 값들 모두 출력

    def Expectation(self):          # E-step
        self.cluster = [[],[]]          # cluster 초기화   (매번 초기화시켜줘야 한다.)
        for x in self.x_list:           # 모든 점들의 값들을 돌며 대입시켜준다.
            c1up = self.mu_sigma_prob[0][2] * self.c1.pdf(x)        # P(c1) * P(x/c1)
            c2up = self.mu_sigma_prob[1][2] * self.c2.pdf(x)        # P(c2) * P(x/c2)
            down = self.mu_sigma_prob[0][2] * self.c1.pdf(x) + self.mu_sigma_prob[1][2] * self.c2.pdf(x)        # P(c1) * P(x/c1) + P(c2) * P(x/c2)

            if c1up/down > c2up/down :          # P(c1) * P(x/c1)  /  (P(c1) * P(x/c1) + P(c2) * P(x/c2))   >   P(c2) * P(x/c2)  /   (P(c1) * P(x/c1) + P(c2) * P(x/c2))
                self.cluster[0].append(x)       # cluster[0]번째 인덱스에 저장한다.
            else:                               # 반대의 경우에는
                self.cluster[1].append(x)       # cluster[1]번쨰 인덱스에 저장한다.

        print("cluster 1:", self.cluster[0])        # cluster 0 번째 인덱스 프린트
        print("cluster 2:", self.cluster[1])        # cluster 1 번째 인덱스 프린트


    def Maximization(self):
        c_list = self.c1 , self.c2          # 정규분푼포의 각 객체를 c_list에 저장한다.
        for i, c in enumerate(c_list):      # 정규분포의 객체(self.c1, self.c2)를 넘버링(enumerate)하여 for문을 돈다.
            sum = 0                         # summation P(c/x) 를 계산할 변수 초기화
            x_mul_sum = 0                   # summation x * P(c/x)를 계산할 변수 초기화
            sigma_sum = 0                   # summation (x-M)**2 * P(c/x)를 계산할 변수 초기화
            for x in self.x_list:           # 모든 점(20개)에 대해서 입력시키며 평균, 표준편차, 가중치 업데이트   참고) 클러스터링 된 점만 도는 K-Means와는 다르다.
                p = ( self.mu_sigma_prob[i][2] * c.pdf(x) ) / (self.mu_sigma_prob[0][2] * self.c1.pdf(x) + self.mu_sigma_prob[1][2] * self.c2.pdf(x) )      # P(c/x)
                sum += p                                                     # summation P(c/x)
                x_mul_sum += x * p                                           # summation x * P(c/x)
                sigma_sum += (x - self.mu_sigma_prob[i][0]) ** 2 * p         # summation (x-M)** 2 * P(c/x)
            prob = sum / len(self.x_list)                                    # summation P(c/x) / ( 1 / n )
            mu = x_mul_sum / sum                                             # summation x * P(c/x) / summation P(c/x)
            sigma = sigma_sum / sum                                          # summation (x-M)** 2 * P(c/x)  /  summation P(c/x)
            self.mu_sigma_prob[i][0] = mu           # 정규분포에 대해 평균 업데이트
            self.mu_sigma_prob[i][1] = sigma        # 정규분포에 대해 표준편차 업데이트
            self.mu_sigma_prob[i][2] = prob         # 정규분포에 대해 가중치 업데이트
        for i in range(2):
            print('P(c{0}) = {1}'.format(i+1, self.mu_sigma_prob[i][2]))            # 업데이트 된 가중치 프린트
            print('c{0} 평균 = {1}'.format(i+1, self.mu_sigma_prob[i][0]))            # 업데이트 된 평균 프린트
            print('c{0} 표준편차 = {1}'.format(i+1, self.mu_sigma_prob[i][1]))          # 업데이트 된 표준편차 프린트


if __name__ == '__main__':      # 메인문
    x1_list = np.random.normal(7, 3, size=10)   # x1_list 에 평균 = 7, 표준편차 = 3, 데이터개수 = 10 인 정규분포에서 랜덤으로 10개를 뽑아 저장한다.
    x2_list = np.random.normal(20, 3, size=10)  # x2_list 에 평균 = 20, 표준편차 = 3, 데이터개수 = 10 인 정규분포에서 랜덤으로 10개를 뽑아 저장한다.
    EmCluster = EM(x1_list,x2_list)         # x1_list와 x2_list를 class EM의 인자값으로 넣는다.
    for i in range(50):     # 50번 반복한다. i : 0~49
        print()     # 한 줄 띈다.
        print('#클러스터', str(i+1)+'#')
        EmCluster.Expectation()         # E-step
        print()
        print('#갱신', str(i + 1) + '#')
        EmCluster.Maximization()        # M-stpe


모든 점들의 좌표 [ 7.47959811  7.96724499  9.72841867  3.25753468  4.43942979  7.22902746
  4.49489602  2.19280597  4.27718386  7.62024041 23.23672941 23.18072875
 26.02614643 21.48459851 18.54422844 25.68888925 22.40494988 19.58429504
 18.63677011 22.05577033]

#클러스터 1#
cluster 1: [7.4795981127859115, 7.967244991533187, 9.728418674963654, 3.2575346811728347, 4.439429789588844, 7.229027463282449, 4.494896024319013, 2.1928059736956182, 4.2771838559436075, 7.620240409134958]
cluster 2: [23.23672941218731, 23.1807287526941, 26.026146428399564, 21.484598509662938, 18.544228441919334, 25.688889251658914, 22.404949877962483, 19.584295041449458, 18.63677010691664, 22.05577032966567]

#갱신 1#
P(c1) = 0.4998167043710489
c1 평균 = 5.868705583754455
c1 표준편차 = 6.66005233029908
P(c2) = 0.5001834158132743
c2 평균 = 22.078298182096855
c2 표준편차 = 10.617388803035414

#클러스터 2#
cluster 1: [7.4795981127859115, 7.967244991533187, 9.728418674963654, 3.2575346811728347, 4.439429789588844, 7.229027463282449, 4.49489602431

P(c1) = 0.49981646370652627
c1 평균 = 5.868702891388689
c1 표준편차 = 5.38020530339112
P(c2) = 0.5001835362934738
c2 평균 = 22.078295624367097
c2 표준편차 = 6.298094866384447

#클러스터 16#
cluster 1: [7.4795981127859115, 7.967244991533187, 9.728418674963654, 3.2575346811728347, 4.439429789588844, 7.229027463282449, 4.494896024319013, 2.1928059736956182, 4.2771838559436075, 7.620240409134958]
cluster 2: [23.23672941218731, 23.1807287526941, 26.026146428399564, 21.484598509662938, 18.544228441919334, 25.688889251658914, 22.404949877962483, 19.584295041449458, 18.63677010691664, 22.05577032966567]

#갱신 16#
P(c1) = 0.49981646370652627
c1 평균 = 5.868702891388689
c1 표준편차 = 5.38020530339112
P(c2) = 0.5001835362934738
c2 평균 = 22.078295624367097
c2 표준편차 = 6.298094866384447

#클러스터 17#
cluster 1: [7.4795981127859115, 7.967244991533187, 9.728418674963654, 3.2575346811728347, 4.439429789588844, 7.229027463282449, 4.494896024319013, 2.1928059736956182, 4.2771838559436075, 7.620240409134958]
cluster 2: [23.236729412

cluster 1: [7.4795981127859115, 7.967244991533187, 9.728418674963654, 3.2575346811728347, 4.439429789588844, 7.229027463282449, 4.494896024319013, 2.1928059736956182, 4.2771838559436075, 7.620240409134958]
cluster 2: [23.23672941218731, 23.1807287526941, 26.026146428399564, 21.484598509662938, 18.544228441919334, 25.688889251658914, 22.404949877962483, 19.584295041449458, 18.63677010691664, 22.05577032966567]

#갱신 34#
P(c1) = 0.49981646370652627
c1 평균 = 5.868702891388689
c1 표준편차 = 5.38020530339112
P(c2) = 0.5001835362934738
c2 평균 = 22.078295624367097
c2 표준편차 = 6.298094866384447

#클러스터 35#
cluster 1: [7.4795981127859115, 7.967244991533187, 9.728418674963654, 3.2575346811728347, 4.439429789588844, 7.229027463282449, 4.494896024319013, 2.1928059736956182, 4.2771838559436075, 7.620240409134958]
cluster 2: [23.23672941218731, 23.1807287526941, 26.026146428399564, 21.484598509662938, 18.544228441919334, 25.688889251658914, 22.404949877962483, 19.584295041449458, 18.63677010691664, 22.0557703